# Práctica 2: GPU Programming (CUDA)

## Preparación del entorno

Eliminación de datos innecesarios creador por Google Collab

In [ ]:
!rm -rf sample_data

Descarga de `Numba` en caso de no encontrarse en el sistema

In [ ]:
!pip install numba --upgrade

Paquetes necesarios para el correcto funcionamiento del código

In [ ]:
import numpy as np
from numba import cuda
import time
import math

## 2.1 Compulsory assignment #1: Matrix transpose

In [ ]:
Ax = 5
Ay = 7
Bx = Ay
By = Ax

def create_matrix_1(Ax: int = 5, Ay: int = 7) -> tuple:
  return np.random.rand(Ax, Ay), np.zeros((Ay, Ax))

In [ ]:
@cuda.jit
def transpose_parallel(A, B):
    i, j = cuda.grid(2)

    if i < B.shape[0] and j < B.shape[1]:
        B[i, j] = A[j, i]

def transpose_sequential(A, B):
    for i in range(0, B.shape[0]):
        for j in range(0, B.shape[1]):
            B[i, j] = A[j, i]

In [ ]:
A, B_seq = create_matrix_1()

t_start = time.time()
transpose_sequential(A, B_seq)
t_finish = time.time()

t_cpu = t_finish - t_start

print(f"A = {A}")
print(f"B sequential = {B_seq}")
print(f"Tiempo ejecución cpu = {t_cpu}")

A = [[0.59519143 0.35179901 0.88770519 0.85486685 0.38753975 0.98115259
  0.49717322]
 [0.34034105 0.31260797 0.0868682  0.26890184 0.73219868 0.1489382
  0.5872679 ]
 [0.77548061 0.55378198 0.5088211  0.17974199 0.22764901 0.68396361
  0.82230342]
 [0.8567986  0.6150403  0.76329363 0.58079591 0.3485574  0.39368057
  0.51019182]
 [0.60263113 0.27987086 0.68787657 0.56656794 0.18172897 0.61801314
  0.71105428]]
B sequential = [[0.59519143 0.34034105 0.77548061 0.8567986  0.60263113]
 [0.35179901 0.31260797 0.55378198 0.6150403  0.27987086]
 [0.88770519 0.0868682  0.5088211  0.76329363 0.68787657]
 [0.85486685 0.26890184 0.17974199 0.58079591 0.56656794]
 [0.38753975 0.73219868 0.22764901 0.3485574  0.18172897]
 [0.98115259 0.1489382  0.68396361 0.39368057 0.61801314]
 [0.49717322 0.5872679  0.82230342 0.51019182 0.71105428]]
Tiempo ejecución cpu = 0.00013113021850585938


In [ ]:
#Explicaciones pal iñigo

_, B_par = create_matrix_1()

# Esto sirve para meter el la vram de la gpu las matrices
A_device = cuda.to_device(A)
B_device = cuda.to_device(B_par)

# Para el uso optimo de la gpu, tenemos que tener en cuenta que
# para maxima eficiencia de un warp (conjunto de hilos) este tiene
# que tener 32, como estamos tratando datos en 2D, dividimos esos
# 32 hilos en dos
threads_per_block = (16, 16)
# Para determinar el número de bloques que debemos usar simplemente
# dividimos el eje entre el número de hilos del eje aplicando la función
# techo para que no se queden posiciones libres sin hilos asignados
# (aunque para ello debamos crear hilos que no se vayan a usar)
blocks_X = math.ceil(B_par.shape[0] / threads_per_block[0])
blocks_Y = math.ceil(B_par.shape[1] / threads_per_block[1])
blocks_total = (blocks_X, blocks_Y)

t_start = time.time()
# Llamamos a la función de la transpuesta en paralelo, para ello, hay
# que indicar el número de bloques que se van a usar en cada dimensión,
# junto al número de hilos que va a tener cada bloque
transpose_parallel[blocks_total, threads_per_block](A_device, B_device)

# Esto es MUY importante poque como lo que acabamos de hacer es "una
# pelea salvaje en la jungla" por los calculos de los hilos, debemos
# asegurarnos que todos acaban, para eso sirve esta función, en caso de
# que un hilo falte por realizar sus calculos no se continuará.
cuda.synchronize()
t_finish = time.time()

# Tras realizar todos los calculos necesarios, se copia de la vram de la gpu
# a la ram de la cpu
B_par = B_device.copy_to_host()

t_gpu = t_finish - t_start

print(f"A = {A}")
print(f"B parallel = {B_par}")
print(f"Tiempo ejecución gpu = {t_gpu}")

A = [[0.59519143 0.35179901 0.88770519 0.85486685 0.38753975 0.98115259
  0.49717322]
 [0.34034105 0.31260797 0.0868682  0.26890184 0.73219868 0.1489382
  0.5872679 ]
 [0.77548061 0.55378198 0.5088211  0.17974199 0.22764901 0.68396361
  0.82230342]
 [0.8567986  0.6150403  0.76329363 0.58079591 0.3485574  0.39368057
  0.51019182]
 [0.60263113 0.27987086 0.68787657 0.56656794 0.18172897 0.61801314
  0.71105428]]
B parallel = [[0.59519143 0.34034105 0.77548061 0.8567986  0.60263113]
 [0.35179901 0.31260797 0.55378198 0.6150403  0.27987086]
 [0.88770519 0.0868682  0.5088211  0.76329363 0.68787657]
 [0.85486685 0.26890184 0.17974199 0.58079591 0.56656794]
 [0.38753975 0.73219868 0.22764901 0.3485574  0.18172897]
 [0.98115259 0.1489382  0.68396361 0.39368057 0.61801314]
 [0.49717322 0.5872679  0.82230342 0.51019182 0.71105428]]
Tiempo ejecución gpu = 0.0005588531494140625


`Pal iñigo: Como puedes darte cuenta, el tiempo de ejecución de la gpu es más alto que el de la cpu cuando por lógica debería ser al revés al hacerlo en paralelo, esto se debe a que al ser un tamaño de matrix pequeño, es mayor el peso de creación de hilos y ponerlos en ejecución que el de la propia ejecución de la traspuesta. Si se decidiese poner un tamaño mucho más grande de matriz se podría ver como esto se cumple.`

In [ ]:
speedup = t_cpu / t_gpu

print(f"Speedup = {speedup}")

Speedup = 0.23464163822525597


## 2.2 Compulsory assignment #2: Average Rows/Cols I

In [ ]:
def create_matrix_2(Ax: int, Ay: int):
    A = np.random.rand(Ax, Ay)
    B = np.zeros(Ay)
    return A, B

def Avg_Rows(input, output):
    for y in range(input.shape[1]):
        output[y] = 0.0
        for x in range(input.shape[0]):
            output[y] += input[x, y]
        output[y] /= input.shape[0]

@cuda.jit
def Avg_Rows_parallel(A, B, Ax):
    y = cuda.grid(1)
    if y < B.shape[0]:
        temp_sum = 0.0
        for x in range(Ax):
            temp_sum += A[x, y]
        B[y] = temp_sum / Ax

In [ ]:
# Pal iñigo: haz tu esto según los comentarios que te he puesto en el ejercicio anterior